In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../Data/Day 05.txt', header=None, names=['input'], skip_blank_lines=False)
df

In [ ]:
seeds = df.input[0]
seeds = seeds.replace('seeds: ','').split()
seeds = list(map(int, seeds))
seeds

In [ ]:
df_noseeds = df.drop([0])
df_noseeds

In [ ]:
# Blank Rows to delineate brea
blank_rows = df_noseeds.loc[pd.isna(df_noseeds['input']), :].index.tolist()
blank_rows

In [ ]:
title_rows = list(map(int, blank_rows))
title_rows = [x+1 for x in title_rows]
title_rows

In [ ]:
titles = df_noseeds.input[title_rows]
titles = titles.str.replace(' map:','')
titles

In [ ]:
mat_start = [x+1 for x in title_rows]
mat_start

In [ ]:
mat_end = [x-1 for x in blank_rows]
mat_end = mat_end[1:]
mat_end.append(len(df_noseeds))
mat_end

In [ ]:
df_meta = pd.DataFrame(
    {
        'title': titles,
        'mat_start': mat_start,
        'mat_end': mat_end
    }
)
df_meta = df_meta.reset_index(drop=True)
df_meta

In [ ]:
map_dict = {}
for index, row in df_meta.iterrows():
    
    # Get Meta values
    title = row['title']
    start = row['mat_start']
    end = row['mat_end']

    df = pd.DataFrame(df_noseeds.input[start:end])
    # df = pd.DataFrame(df)
    df = df.input.str.split(expand = True)
    df.rename(columns={0:'dest_start', 1:'source_start', 2:'length'}, inplace=True)
    
    map_dict[title] = df.reset_index(drop=True)

    # get full sequences to map
    map_dict[title]['source_range'] = map_dict[title].apply(lambda  row: range(int(row['source_start']), int(row['source_start'])+int(row['length'])), axis=1)
    map_dict[title]['dest_range'] = map_dict[title].apply(lambda  row: range(int(row['dest_start']), int(row['dest_start'])+int(row['length'])), axis=1)
    

In [ ]:
def convert_source_to_dest(source, mat):

    # into which row of the matrix does the source value fall?
    found_source = list(mat.apply(lambda row: source in row['source_range'], axis=1))
    row = np.where(found_source)[0]

    if len(row)==0:
        dest = source
    else:
        # Find actual source number in source range series
        element = source - int(mat.source_start[row].iloc[0])
        
        # Calculate destination number from dest range series
        dest = int(mat.dest_start[row].iloc[0]) + element
    
    return(dest)

In [ ]:
df_final = pd.DataFrame({'seed':seeds})
df_final['soil'] = df_final.apply(lambda row: convert_source_to_dest(source = int(row['seed']), mat=map_dict['seed-to-soil']), axis=1)
df_final['fertilizer'] = df_final.apply(lambda row: convert_source_to_dest(source = int(row['soil']), mat=map_dict['soil-to-fertilizer']), axis=1)
df_final['water'] = df_final.apply(lambda row: convert_source_to_dest(source = int(row['fertilizer']), mat=map_dict['fertilizer-to-water']), axis=1)
df_final['light'] = df_final.apply(lambda row: convert_source_to_dest(source = int(row['water']), mat=map_dict['water-to-light']), axis=1)
df_final['temperature'] = df_final.apply(lambda row: convert_source_to_dest(source = int(row['light']), mat=map_dict['light-to-temperature']), axis=1)
df_final['humidity'] = df_final.apply(lambda row: convert_source_to_dest(source = int(row['temperature']), mat=map_dict['temperature-to-humidity']), axis=1)
df_final['location'] = df_final.apply(lambda row: convert_source_to_dest(source = int(row['humidity']), mat=map_dict['humidity-to-location']), axis=1)
df_final

In [ ]:
min(df_final['location'])